In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/yuliz/Downloads/Aviana/train_set.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns

Index(['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', 'Model_Year',
       'Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12', 'OrdCat', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6',
       'Var7', 'Var8', 'NVCat', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4',
       'Claim_Amount'],
      dtype='object')

In [4]:
#function to get number of missing values in a column
def get_na_num(column): #input the whole column
    return column.isnull().sum() + column[column == ''].count() + column[column == '?'].count()

In [5]:
#function to get number of valid values in a column
def get_valid_num(column): #input the whole column
    return len(column) - get_na_num(column)

In [6]:
#function to get minimum value in a column
def get_min(column): #input the whole column
    return min(column)

In [7]:
#function to get maximum value in a column
def get_max(column): #input the whole column
    return max(column)

In [8]:
#function to get mean in a column
def get_mean(column): #input the whole column
    return column.mean()

In [9]:
#function to get std in a column
def get_std(column): #input the whole column
    return column.std()

In [10]:
#function to get skewness in a column
def get_skew(column): #input the whole column
    return column.skew()

In [11]:
#function to get number of distict values in a column
def get_distinct_num(column): #input the whole column
    return len(column.unique().tolist())

In [12]:
#function to get count of each distict value in a column
def get_distinct_count(column): #input the whole column
    if get_distinct_num(column) > 5:
        print('Number of distict values is larger than 5. We stop updating the number of distinct values')
    else:
        return column.value_counts()

In [13]:
#function to get median in a column
import statistics
def get_median(column):
    if get_distinct_num(column) > 5:
        print('Number of distict values is larger than 5. We do not calculate median')
    else:
        return statistics.median(column)

In [14]:
#function to get mode and count for the mode in a column
def get_mode(column):
    return (column.mode()[0],column[column==column.mode()[0]].count())

In [15]:
#function to check whether a numerical column is continuous or discrete
def check_cont_or_dis(column,threshold):
    if get_distinct_num(column)/get_valid_num(column) > float(threshold): #if relative is bigger than the threshold, we consider it to be continuous 
        return ('continuous')
    else:
        return ('discrete')

In [53]:
# funtion to distinguish variable's type from continuous, ordinal, categorical ordinal and nominal
def column_type(column, threshold):
    if column.dtype == np.float64: #float type
        return ('continuous')
    elif column.dtype == np.int64: #int type
        if check_cont_or_dis(column,threshold) == 'discrete' and get_distinct_num(column) <= 15:
            return ('ordinal')
        else: 
            return ('continuous')
    else: #string type and others
        class1 = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', "I", 'J', 'K', 'L', 'M', 'N', 'MISSING']
        class2 = ['SSSS', 'SSS', 'SS', 'S', 'A', 'B', 'C', 'D', "E", 'F', 'G', 'H', "I", 'J', 'MISSING']
        unique_value = [str(x).upper() for x in column.unique().tolist()]
        count_class1 = 0 
        count_class2 = 0
        for i in unique_value:
            if (i in class1):
                count_class1 += 1
            elif (i in class2):
                count_class1 += 1
            else:
                count_class1 = 0
                count_class2 = 0
        if ((count_class1 <= 15 and count_class1 != 0) or (count_class2 <= 15 and count_class2 != 0)):
            return ('cat_ordinal')
        else:
            return ('nominal')

In [20]:
#function to do basic variable screening and create basic statistical report
def Stats_Collection(df):
    threshold = input('Threshold to determine continuous or discrete: ')
    for c in df:
        print('Variable name: ',c)
        
        #Basic variable screening
        if get_na_num(df[c])/len(df[c]) > 0.5:
            print('More 50% missing values, drop this column\n')
            df = df.drop(columns=c)
            continue
        if (column_type(df[c],threshold) == 'continuous') and (get_min(df[c]) == get_max(df[c])):
            print('All same value, drop this column\n')
            df = df.drop(columns=c)
            continue
        if (column_type(df[c],threshold) == 'ordinal' or column_type(df[c],threshold) == 'cat_ordinal'or column_type(df[c],threshold) == 'nominal') and (get_mode(df[c])[1]/get_valid_num(df[c]) > 0.95):
            print('Mode contains more than 95% cases, drop this column\n')
            df = df.drop(columns=c)
            continue
        if (column_type(df[c],threshold) == 'nominal') and (get_distinct_num(df[c]) > 100):
            print('More than 100 categories, drop this column\n')
            df = df.drop(columns=c)
            continue
            
        #Basic statistic report
        print('Variable type: ', column_type(df[c],threshold))
        print ('Number of missing values: ',get_na_num(df[c]))
        print ('Number of valid values: ',get_valid_num(df[c]))
        if column_type(df[c],threshold) == 'continuous' or column_type(df[c],threshold) == 'ordinal':
            print('Minimum value: ', get_min(df[c]))
            print('Maximum value: ', get_max(df[c]))
            print('Mean: ',get_mean(df[c]))
            print('Standard Deviation: ',get_std(df[c]))
            print('Skewness: ',get_skew(df[c]))
        if column_type(df[c],threshold) == 'continuous':
            print('Number of distinct values: ',get_distinct_num(df[c]))
            print('Number of cases for each distinct value: ')
            print(get_distinct_count(df[c]))
        else:
            print('Number of categories: ', get_distinct_num(df[c]))
            print('The counts of each category: ')
            print(get_distinct_count(df[c]))
            print('Mode: ', get_mode(df[c])[0],'Count: ',get_mode(df[c])[1])    
        print()

In [54]:
df_sample = df.iloc[:1000,]
df_variable = df_sample.drop(columns = ['Claim_Amount'])
Stats_Collection(df_variable)

Threshold to determine continuous or discrete: 0.95
Variable name:  Row_ID
Variable type:  continuous
Number of missing values:  0
Number of valid values:  1000
Minimum value:  1
Maximum value:  1000
Mean:  500.5
Standard Deviation:  288.8194360957494
Skewness:  0.0
Number of distinct values:  1000
Number of cases for each distinct value: 
Number of distict values is larger than 5. We stop updating the number of distinct values
None

Variable name:  Household_ID
Variable type:  continuous
Number of missing values:  0
Number of valid values:  1000
Minimum value:  1
Maximum value:  576
Mean:  270.473
Standard Deviation:  168.96537683670454
Skewness:  0.13733710613930392
Number of distinct values:  466
Number of cases for each distinct value: 
Number of distict values is larger than 5. We stop updating the number of distinct values
None

Variable name:  Vehicle
Variable type:  ordinal
Number of missing values:  0
Number of valid values:  1000
Minimum value:  1
Maximum value:  9
Mean:  2.1

Variable type:  cat_ordinal
Number of missing values:  0
Number of valid values:  1000
Number of categories:  14
The counts of each category: 
Number of distict values is larger than 5. We stop updating the number of distinct values
None
Mode:  M Count:  434

Variable name:  NVVar1
Variable type:  continuous
Number of missing values:  0
Number of valid values:  1000
Minimum value:  -0.2315299
Maximum value:  3.1977898999999996
Mean:  -0.06989462069999998
Standard Deviation:  0.5989492424052759
Skewness:  3.712229321132413
Number of distinct values:  5
Number of cases for each distinct value: 
-0.231530    928
 2.054683     35
 1.483130     26
 3.197790     10
 2.511926      1
Name: NVVar1, dtype: int64

Variable name:  NVVar2
Variable type:  continuous
Number of missing values:  0
Number of valid values:  1000
Minimum value:  -0.2661168
Maximum value:  4.3084822
Mean:  -0.01481881789999995
Standard Deviation:  0.8400836252229962
Skewness:  3.339145083997441
Number of distinct values:  